# Inception-v4

In [ ]:
%conda install -y gdown

In [ ]:
import gdown
import zipfile
from pathlib import Path

from tensorflow.config import list_physical_devices
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Input, Layer, Conv2D, MaxPool2D, AvgPool2D, Rescaling, GlobalAveragePooling2D, Dropout, Flatten, Dense, concatenate
from tensorflow.keras import Model

In [ ]:
zip_name = "dataset.zip"
wd = Path("/kaggle/working")
extract_path = Path(wd, "data")
class_names = list()

train_path = Path(extract_path, "train")
test_path = Path(extract_path, "test")
model_path = Path(wd, "models/final_models/inception4.h5")

## Model settings

In [ ]:
batch_size = 64
img_size = 299
epochs = 100
seed = 27
validation_split = 0.2

## Download dataset

In [ ]:
gdown.download(
    "https://drive.google.com/uc?id=18_MDbhjncjKGwa1N9zYYUmiFzZYxzF-6",
    zip_name
)

zip_ref = zipfile.ZipFile(Path(wd, zip_name), 'r')
zip_ref.extractall(extract_path)
zip_ref.close()

## Get class names from the training directory

In [ ]:
class_names = [class_name.name for class_name in train_path.iterdir()]
for class_name in class_names:
    print(class_name)

## Check number of GPUs

In [ ]:
print("Num GPUs Available: {}".format(len(list_physical_devices('GPU'))))

## Load training and validation data

In [ ]:
training_data = image_dataset_from_directory(
    directory=train_path,
    validation_split=validation_split,
    subset='training',
    labels='inferred',
    class_names=class_names,
    label_mode='int',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    seed=seed,
    shuffle=True
)

validation_data = image_dataset_from_directory(
    directory=train_path,
    validation_split=validation_split,
    subset='validation',
    labels='inferred',
    class_names=class_names,
    label_mode='int',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    seed=seed,
    shuffle=True
)

## Define Inception-v4 model parts

In [ ]:
def Stem(inputs):
    x = Conv2D(filters=32, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')(inputs)
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='valid', activation='relu')(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(x)

    x0 = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)
    x1 = Conv2D(filters=96, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')(x)

    x = concatenate([x0, x1], axis=-1)

    x0 = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')(x)
    x0 = Conv2D(filters=96, kernel_size=(3, 3), padding='valid', activation='relu')(x0)

    x1 = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')(x)
    x1 = Conv2D(filters=64, kernel_size=(7, 1), padding='same', activation='relu')(x1)
    x1 = Conv2D(filters=64, kernel_size=(1, 7), padding='same', activation='relu')(x1)
    x1 = Conv2D(filters=96, kernel_size=(3, 3), padding='valid', activation='relu')(x1)

    x = concatenate([x0, x1], axis=-1)

    x0 = Conv2D(filters=192, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')(x)
    x1 = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

    return concatenate([x0, x1], axis=-1)

In [ ]:
def InceptionA(inputs):
    x0 = AvgPool2D(pool_size=(3, 3), strides=(1, 1), padding='same')(inputs)
    x0 = Conv2D(filters=96, kernel_size=(1, 1), padding='same', activation='relu')(x0)

    x1 = Conv2D(filters=96, kernel_size=(1, 1), padding='same', activation='relu')(inputs)

    x2 = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')(inputs)
    x2 = Conv2D(filters=96, kernel_size=(3, 3), padding='same', activation='relu')(x2)

    x3 = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')(inputs)
    x3 = Conv2D(filters=96, kernel_size=(3, 3), padding='same', activation='relu')(x3)
    x3 = Conv2D(filters=96, kernel_size=(3, 3), padding='same', activation='relu')(x3)

    return concatenate([x0, x1, x2, x3], axis=-1)


In [ ]:
def InceptionB(inputs):
    x0 = AvgPool2D(pool_size=(3, 3), strides=(1, 1), padding='same')(inputs)
    x0 = Conv2D(filters=128, kernel_size=(1, 1), padding='same', activation='relu')(x0)

    x1 = Conv2D(filters=384, kernel_size=(1, 1), padding='same', activation='relu')(inputs)

    x2 = Conv2D(filters=192, kernel_size=(1, 1), padding='same', activation='relu')(inputs)
    x2 = Conv2D(filters=224, kernel_size=(1, 7), padding='same', activation='relu')(x2)
    x2 = Conv2D(filters=256, kernel_size=(1, 7), padding='same', activation='relu')(x2)

    x3 = Conv2D(filters=192, kernel_size=(1, 1), padding='same', activation='relu')(inputs)
    x3 = Conv2D(filters=192, kernel_size=(1, 7), padding='same', activation='relu')(x3)
    x3 = Conv2D(filters=224, kernel_size=(7, 1), padding='same', activation='relu')(x3)
    x3 = Conv2D(filters=224, kernel_size=(1, 7), padding='same', activation='relu')(x3)
    x3 = Conv2D(filters=256, kernel_size=(7, 1), padding='same', activation='relu')(x3)

    return concatenate([x0, x1, x2, x3], axis=-1)

In [ ]:
def InceptionC(inputs):
    x0 = AvgPool2D(pool_size=(3, 3), strides=(1, 1), padding='same')(inputs)
    x0 = Conv2D(filters=256, kernel_size=(1, 1), padding='same', activation='relu')(x0)

    x1 = Conv2D(filters=256, kernel_size=(1, 1), padding='same', activation='relu')(inputs)

    x2 = Conv2D(filters=384, kernel_size=(1, 1), padding='same', activation='relu')(inputs)
    x2_0 = Conv2D(filters=256, kernel_size=(1, 3), padding='same', activation='relu')(x2)
    x2_1 = Conv2D(filters=256, kernel_size=(3, 1), padding='same', activation='relu')(x2)
    x2 = concatenate([x2_0, x2_1], axis=-1)

    x3 = Conv2D(filters=384, kernel_size=(1, 1), padding='same', activation='relu')(inputs)
    x3 = Conv2D(filters=448, kernel_size=(1, 3), padding='same', activation='relu')(x3)
    x3 = Conv2D(filters=512, kernel_size=(3, 1), padding='same', activation='relu')(x3)
    x3_0 = Conv2D(filters=256, kernel_size=(3, 1), padding='same', activation='relu')(x3)
    x3_1 = Conv2D(filters=256, kernel_size=(1, 3), padding='same', activation='relu')(x3)
    x3 = concatenate([x3_0, x3_1], axis=-1)

    return concatenate([x0, x1, x2, x3], axis=-1)

In [ ]:
def ReductionA(inputs):
    x0 = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(inputs)

    x1 = Conv2D(filters=384, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')(inputs)

    x2 = Conv2D(filters=192, kernel_size=(1, 1), padding='same', activation='relu')(inputs)
    x2 = Conv2D(filters=224, kernel_size=(3, 3), padding='same', activation='relu')(x2)
    x2 = Conv2D(filters=256, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')(x2)

    return concatenate([x0, x1, x2], axis=-1)

In [ ]:
def ReductionB(inputs):
    x0 = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(inputs)

    x1 = Conv2D(filters=192, kernel_size=(1, 1), padding='same', activation='relu')(inputs)
    x1 = Conv2D(filters=192, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')(x1)

    x2 = Conv2D(filters=256, kernel_size=(1, 1), padding='same', activation='relu')(inputs)
    x2 = Conv2D(filters=256, kernel_size=(1, 7), padding='same', activation='relu')(x2)
    x2 = Conv2D(filters=320, kernel_size=(7, 1), padding='same', activation='relu')(x2)
    x2 = Conv2D(filters=320, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu')(x2)

    return concatenate([x0, x1, x2], axis=-1)

In [ ]:
def Inception4():
    inputs = Input(shape=(img_size, img_size, 3))

    x = Stem(inputs)

    for _ in range(4):
        x = InceptionA(x)

    x = ReductionA(x)

    for _ in range(7):
        x = InceptionB(x)

    x = ReductionB(x)
    
    for _ in range(3):
        x = InceptionC(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(rate=0.2)(x)
    x = Flatten()(x)
    x = Dense(units=len(class_names), activation='softmax')(x)

    return Model(inputs=inputs, outputs=x, name='Inception-v4')

In [ ]:
model = Inception4()

model.compile(
    optimizer=SGD(learning_rate=0.001, momentum=0.9),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

plot_model(
    model, 
    to_file='inception_v4.png', 
    show_shapes=True,
    show_layer_names=True
)